In [1]:
%run ../../Environment/environment.ipynb

e:\Studienarbeit_GitHub\Studienarbeit\Agents\PPO
Notebook ausgeführt


# 1. Bibliotheken importieren

In [2]:
import numpy as np
import pandas as pd
import optuna
import joblib
import random
from stable_baselines3 import PPO, A2C, DQN
from stable_baselines3.common.callbacks import CheckpointCallback
from sklearn.preprocessing import StandardScaler
from stable_baselines3.common.vec_env import DummyVecEnv
import torch
from torch import nn

# 2. Seed setzen

In [3]:
seed = 42
SEED  = seed % (2**32 - 1)
print(f"SEED: {SEED}")

SEED: 42


# 3. Daten einlesen

In [4]:
# -------------------------------
# CSV Datem einlesen
# -------------------------------
train_data = pd.read_csv("../../Transform_data/stand_data/2023-2018_stand_data.csv")
train_data.drop('datetime', axis=1, inplace=True)

test_data = pd.read_csv("../../Transform_data/stand_data/2025-2024_stand_data.csv")
test_data.drop('datetime', axis=1, inplace=True)

if(train_data is not None and test_data is not None):
    print("Daten erfolgreich eingelesen")

Daten erfolgreich eingelesen


# 4. Parallele Umgebungen erstellen für das Training

In [5]:
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.vec_env import VecNormalize

def create_env():
    return TradingEnv(
        data=train_data,
        initial_cash=10_000,
        window_size=336,
        scaler_path="../../Transform_data/scaler.pkl",
        default_seed=SEED
    )

n_envs = 4  # Mehr parallele Umgebungen (8, 16 oder sogar 32 testen!)
env = SubprocVecEnv([create_env for _ in range(n_envs)])

env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.0)
env.training = True  # Sicherstellen, dass Normalisierung aktiv ist

# 5. Hyperparameter Evaluierung

In [6]:
# Für Hyperparameter-Tuning nutzen wir test_data als Validierungsdatensatz.
valid_data = test_data.copy()

# -------------------------------
# Environment-Erstellung
# -------------------------------
def make_env(data):
    def _init():
        return TradingEnv(
            data=data,
            initial_cash=10_000,
            window_size=336,
            scaler_path="../../Transform_data/scaler.pkl",
            default_seed=SEED
        )
    return _init

# -------------------------------
# Evaluation Helper Function
# -------------------------------
def evaluate_agent(model, env, n_eval_episodes=5):
    episode_rewards = []
    for _ in range(n_eval_episodes):
        obs = env.reset()
        done = False
        total_reward = 0.0
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, _ = env.step(action)
            total_reward += reward
        episode_rewards.append(total_reward)
    return np.mean(episode_rewards)

In [7]:
# -------------------------------
# Hyperparameter Tuning with Optuna
# -------------------------------
def objective(trial):
    # Hyperparameter-Sampling
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    gamma = trial.suggest_float("gamma", 0.90, 0.9999)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256])
    n_steps = trial.suggest_categorical("n_steps", [128, 256, 512])  # keine 1024
    ent_coef = trial.suggest_float("ent_coef", 1e-6, 0.01, log=True)
    clip_range = trial.suggest_float("clip_range", 0.1, 0.4)
    gae_lambda = trial.suggest_float("gae_lambda", 0.8, 0.99)

    # Train-Environment mit SubprocVecEnv und VecNormalize
    n_envs = 4
    env_train_raw = SubprocVecEnv([make_env(train_data) for _ in range(n_envs)])
    env_train = VecNormalize(env_train_raw, norm_obs=True, norm_reward=True, clip_obs=10.0)
    env_train.training = True

    # Validation-Environment mit DummyVecEnv (nur 1 Env)
    env_valid_raw = DummyVecEnv([make_env(valid_data)])
    env_valid = VecNormalize(env_valid_raw, norm_obs=True, norm_reward=True, clip_obs=10.0)
    env_valid.training = False
    env_valid.norm_reward = False

    # PPO Agent
    model = PPO(
        "MlpPolicy",
        env_train,
        learning_rate=learning_rate,
        gamma=gamma,
        batch_size=batch_size,
        n_steps=n_steps,
        ent_coef=ent_coef,
        clip_range=clip_range,
        gae_lambda=gae_lambda,
        verbose=0,
        seed=SEED,
        policy_kwargs=dict(
            net_arch=dict(pi=[128, 128], vf=[128, 128]),
            activation_fn=nn.ReLU,
        )
    )

    # Training
    model.learn(total_timesteps=10_000, log_interval=1)

    # Evaluation
    mean_reward = evaluate_agent(model, env_valid, n_eval_episodes=5)

    return mean_reward

In [8]:
# -------------------------------
# Optuna-Optimierung starten
# -------------------------------
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print("Best hyperparameters:", study.best_trial.params)

[I 2025-04-06 19:59:07,717] A new study created in memory with name: no-name-3c8384b9-49ac-4c5f-911f-eef615846674


Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42


[I 2025-04-06 20:01:20,897] Trial 0 finished with value: 2.9310214519500732 and parameters: {'learning_rate': 2.5641121461446072e-05, 'gamma': 0.9787420180414022, 'batch_size': 64, 'n_steps': 128, 'ent_coef': 1.472739259713409e-06, 'clip_range': 0.15238102512913426, 'gae_lambda': 0.9786570351310979}. Best is trial 0 with value: 2.9310214519500732.


Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42


[I 2025-04-06 20:03:36,783] Trial 1 finished with value: 2.9682915210723877 and parameters: {'learning_rate': 0.0026318231496464234, 'gamma': 0.9121417756631408, 'batch_size': 64, 'n_steps': 256, 'ent_coef': 1.2544509602022392e-05, 'clip_range': 0.1540633153737754, 'gae_lambda': 0.8479590972875454}. Best is trial 1 with value: 2.9682915210723877.


...

Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42


[I 2025-04-06 21:46:30,738] Trial 48 finished with value: 0.0 and parameters: {'learning_rate': 0.006268993671996071, 'gamma': 0.9000374168969688, 'batch_size': 256, 'n_steps': 256, 'ent_coef': 1.6189846336731218e-06, 'clip_range': 0.2118313724736177, 'gae_lambda': 0.8850930703706829}. Best is trial 1 with value: 2.9682915210723877.


Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42


[I 2025-04-06 21:48:56,252] Trial 49 finished with value: 2.9680628776550293 and parameters: {'learning_rate': 1.0161288381546387e-05, 'gamma': 0.9664916228616379, 'batch_size': 32, 'n_steps': 128, 'ent_coef': 5.2432697315807034e-05, 'clip_range': 0.1852565658739072, 'gae_lambda': 0.8286965570074128}. Best is trial 1 with value: 2.9682915210723877.


Seed in the environment: 42
Best hyperparameters: {'learning_rate': 0.0026318231496464234, 'gamma': 0.9121417756631408, 'batch_size': 64, 'n_steps': 256, 'ent_coef': 1.2544509602022392e-05, 'clip_range': 0.1540633153737754, 'gae_lambda': 0.8479590972875454}


Best hyperparameters: {'learning_rate': 0.0026318231496464234, 'gamma': 0.9121417756631408, 'batch_size': 64, 'n_steps': 256, 'ent_coef': 1.2544509602022392e-05, 'clip_range': 0.1540633153737754, 'gae_lambda': 0.8479590972875454}

# 6. Erstellen des Agenten

In [9]:
# Erstelle den PPO-Agenten ohne Hyperparametern
model_without = PPO("MlpPolicy", env, seed=SEED, verbose=1)

Using cpu device


In [14]:
# Mittleres Neuronales Netz
policy_kwargs = dict(
    net_arch=[dict(pi=[128, 128], vf=[128, 128])],  # Zwei Layer mit 128 Neuronen
    activation_fn=nn.ReLU,  # Verwende ReLU als Aktivierungsfunktion
)

# Erstelle den PPO-Agenten mit Optuna-Hyperparametern
model_optuna = PPO(
    "MlpPolicy",
    env,
    learning_rate=0.0026318231496464234,
    gamma=0.9121417756631408,
    batch_size=64,
    n_steps=256,
    ent_coef=1.2544509602022392e-05,
    clip_range=0.1540633153737754,
    gae_lambda=0.8479590972875454,
    policy_kwargs=policy_kwargs,
    verbose=1,
    seed=SEED,
    device="cuda",
)

Using cpu device


In [15]:
# Erstelle den PPO-Agenten mit einem kleineren Netzwerk und Custom parametern
policy_kwargs_small = dict(
    net_arch=[dict(pi=[64, 64], vf=[64, 64])],         # Einfaches Netz
    activation_fn=nn.Tanh,     # Testweise Tanh statt ReLU
)

model_custom_small = PPO(
    "MlpPolicy",
    env,
    learning_rate=0.0003,         # Standard bei SB3
    gamma=0.95,                   # Etwas kürzerer Zeithorizont
    batch_size=64,
    n_steps=512,                  # Etwas mehr Kontext
    ent_coef=0.01,                # Höhere Entropiestrafe → mehr Exploration
    clip_range=0.2,
    gae_lambda=0.92,
    policy_kwargs=policy_kwargs_small,
    verbose=1,
    seed=SEED,
    device="cuda",
)

Using cpu device


In [16]:
# Erstelle den PPO-Agenten mit einem größeren Netzwerk und Custom parametern
policy_kwargs_deep = dict(
    net_arch=[dict(pi=[256, 256, 128], vf=[256, 256, 128])],  # Tieferes Netz
    activation_fn=nn.ReLU,
)

model_custom_deep = PPO(
    "MlpPolicy",
    env,
    learning_rate=0.0001,        # Sehr vorsichtige Lernrate
    gamma=0.99,
    batch_size=128,
    n_steps=1024,                # Längere Rollouts
    ent_coef=0.0001,             # Wenig Exploration
    clip_range=0.25,
    gae_lambda=0.95,
    policy_kwargs=policy_kwargs_deep,
    verbose=1,
    seed=SEED,
    device="cuda",
)

Using cpu device


# 7. Modell trainieren und speichern

In [ ]:
model_without.learn(total_timesteps=200_000)
model_without.save("PPO_Model_Without_200K")

-----------------------------
| time/              |      |
|    fps             | 949  |
|    iterations      | 1    |
|    time_elapsed    | 8    |
|    total_timesteps | 8192 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 501         |
|    iterations           | 2           |
|    time_elapsed         | 32          |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.119216524 |
|    clip_fraction        | 0.626       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.78       |
|    explained_variance   | 0.0557      |
|    learning_rate        | 0.0003      |
|    loss                 | -0.109      |
|    n_updates            | 140         |
|    policy_gradient_loss | -0.056      |
|    value_loss           | 0.00788     |
-----------------------------------------
----------------------------------

In [ ]:
model_optuna.learn(total_timesteps=200_000)
model_optuna.save("PPO_Model_Optuna_200K")

In [ ]:
model_custom_small.learn(total_timesteps=200_000)
model_custom_small.save("PPO_Model_Custom_Small_200K")

In [ ]:
model_custom_deep.learn(total_timesteps=200_000)
model_custom_deep.save("PPO_Model_Custom_Deep_200K")